In [1]:
import pyproj
import math
from pykalman import KalmanFilter
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import read_keyframe_trajectory, read_map_points, read_gps_estimate, kabsch_umeyama, smooth_elevation


keyframes = read_keyframe_trajectory("KeyframeTrajectory.txt", "GPSTrajectory.txt")[1:]  # skip first keyframe (0,0,0)
map_points = read_map_points("MapPoints.txt")
gps_estimate = read_gps_estimate("GPS_estimates.txt")

noisy_elevation = np.array([kf.gps.alt for kf in keyframes])
smoothed_elevation, filtered_elevation = smooth_elevation(noisy_elevation)

kf_trajectory = np.array([[kf.x, kf.y, kf.z] for kf in keyframes])

tr = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
scale_factor = 1 / math.cos(math.radians(np.mean([kf.gps.lon for kf in keyframes])))
gps_path = np.array([tr.transform(kf.gps.lon, kf.gps.lat) for kf in keyframes])
gps_trajectory = np.hstack((gps_path, smoothed_elevation.reshape(-1, 1)))

R, c, t = kabsch_umeyama(gps_trajectory, kf_trajectory)
aligned_kf_trajectory = np.array([t + c * R @ p for p in kf_trajectory])
# t2 = aligned_kf_trajectory[0] - gps_trajectory[0]
# o_aligned_kf_trajectory = aligned_kf_trajectory - t2

fig = go.Figure()

fig.add_trace(go.Scatter(x=gps_trajectory[:, 0], y=gps_trajectory[:, 1], mode='lines', name='GPS', marker=dict(color='red')))
fig.add_trace(go.Scatter(x=aligned_kf_trajectory[:, 0], y=aligned_kf_trajectory[:, 1], mode='lines', name='Aligned KF', marker=dict(color='blue')))
# fig.add_trace(go.Scatter(x=o_aligned_kf_trajectory[:, 0], y=o_aligned_kf_trajectory[:, 1], mode='lines', name='Origin-aligned KF', marker=dict(color='green')))

fig.update_yaxes(scaleanchor="x", scaleratio=scale_factor,)
fig.update_layout(title="GPS vs. KF Trajectory (Mercator)", xaxis_title="X", yaxis_title="Y", margin=dict(l=0, r=0, t=40, b=0))
fig.show()

In [2]:
kf_trajectory = np.array([[kf.x, kf.y, kf.z] for kf in keyframes])
mp_points = np.array([[mp.x, mp.y, mp.z] for mp in map_points])
adjusted_mp_points = np.array([t + c * R @ p for p in mp_points])

adjusted_slam_fig = go.Figure(data=[
    go.Scatter3d(x=gps_trajectory[:, 0], y=gps_trajectory[:, 1], z=gps_trajectory[:, 2],
                 mode='lines', name='GPS trajectory', marker=dict(color='red')),
    go.Scatter3d(x=aligned_kf_trajectory[:, 0], y=aligned_kf_trajectory[:, 1], z=aligned_kf_trajectory[:,
                 2], mode='lines', name='adjusted SLAM trajectory', marker=dict(color='blue')),
    go.Scatter3d(x=adjusted_mp_points[:, 0], y=adjusted_mp_points[:, 1], z=adjusted_mp_points[:, 2], mode='markers', name='adjusted MapPoints', 
                 marker=dict(size=0.5, opacity=0.7, color='black'))])

adjusted_slam_fig.update_layout(title="GPS vs. KF Trajectory (Mercator)", xaxis_title="X", yaxis_title="Y", margin=dict(l=0, r=0, t=40, b=0))

adjusted_slam_fig.show()
